In [22]:
import ee
import wxee
import pandas as pd
import xarray as xr
from datetime import datetime, timedelta
import os

In [23]:
ee.Initialize()

In [24]:
region_polygon = ee.Geometry.Rectangle([-99, 29, -97, 31])

In [25]:
surface_variables = [
    "temperature_2m_above_ground",
    "specific_humidity_2m_above_ground",
    "relative_humidity_2m_above_ground",
    "u_component_of_wind_10m_above_ground",
    "v_component_of_wind_10m_above_ground",
    "precipitable_water_entire_atmosphere"
]

In [26]:
start_date = datetime(2024, 2, 1)
end_date   = datetime(2024, 2, 8)

In [27]:
all_datasets = []
current = start_date

In [28]:
out_dir = "gfs_1_week_data"
os.makedirs(out_dir, exist_ok=True)

In [ ]:
while current < end_date:
    next_day = current + timedelta(days=1)
    
    current_str = current.strftime("%Y-%m-%d")
    next_day_str = next_day.strftime("%Y-%m-%d")
    
    gfs_subset = (
        ee.ImageCollection("NOAA/GFS0P25")
        .filterDate(current_str, next_day_str)
        .filterBounds(region_polygon)
        .select(surface_variables)
        .filterMetadata("forecast_hours", "less_than", 24) 
    )
    
    # Number of Images
    count = gfs_subset.size().getInfo()
    print(f"Date: {current_str}, Images found: {count}")
    
    # If count=0, skip or handle error
    if count == 0:
        print("No data for this day, skipping.")
        current = next_day
        continue
    
    # 6) Convert subset to xarray
    try:
        xr_day = gfs_subset.wx.to_xarray(
            region=region_polygon,
            scale=25000,   # ~25 km
            progress=True,
            num_cores=1,
            masked=True
        )
    except Exception as e:
        print(f"Error downloading data for {current_str}: {e}")
        current = next_day
        continue
    
    # 7) Inspect dataset
    print(xr_day)  # dims, coords, variables
    # e.g., print(xr_day["temperature_2m_above_ground"])
    
    # 8) Save locally as NetCDF
    out_file = os.path.join(out_dir, f"gfs_{current_str}.nc")
    xr_day.to_netcdf(out_file)
    print(f"Saved {out_file}")
    
    # Move to the next day
    current = next_day

Date: 2024-02-01, Images found: 96


Requesting data:   0%|          | 0/96 [03:11<?, ?it/s]
/Users/dhruvyadav/anaconda3/envs/NWS/lib/python3.10/site-packages/wxee/utils.py:149: UserWarning: Different non-null values were encountered for the same variable at the same time coordinate. The first value was taken.
  warnings.warn(


<xarray.Dataset> Size: 19kB
Dimensions:                               (time: 4, x: 10, y: 10)
Coordinates:
  * time                                  (time) datetime64[ns] 32B 2024-02-0...
  * x                                     (x) float64 80B -98.93 ... -96.91
  * y                                     (y) float64 80B 31.1 30.88 ... 29.08
    spatial_ref                           int64 8B 0
Data variables:
    temperature_2m_above_ground           (time, y, x) float64 3kB 16.63 ... nan
    specific_humidity_2m_above_ground     (time, y, x) float64 3kB 0.005454 ....
    relative_humidity_2m_above_ground     (time, y, x) float64 3kB 44.3 ... nan
    u_component_of_wind_10m_above_ground  (time, y, x) float64 3kB -2.418 ......
    v_component_of_wind_10m_above_ground  (time, y, x) float64 3kB 3.056 ... nan
    precipitable_water_entire_atmosphere  (time, y, x) float64 3kB 12.33 ... nan
Attributes:
    TIFFTAG_XRESOLUTION:     1
    TIFFTAG_YRESOLUTION:     1
    TIFFTAG_RESOLUTIONUNIT:  

Requesting data:   0%|          | 0/96 [02:40<?, ?it/s]
/Users/dhruvyadav/anaconda3/envs/NWS/lib/python3.10/site-packages/wxee/utils.py:149: UserWarning: Different non-null values were encountered for the same variable at the same time coordinate. The first value was taken.
  warnings.warn(


<xarray.Dataset> Size: 19kB
Dimensions:                               (time: 4, x: 10, y: 10)
Coordinates:
  * time                                  (time) datetime64[ns] 32B 2024-02-0...
  * x                                     (x) float64 80B -98.93 ... -96.91
  * y                                     (y) float64 80B 31.1 30.88 ... 29.08
    spatial_ref                           int64 8B 0
Data variables:
    temperature_2m_above_ground           (time, y, x) float64 3kB 18.66 ... nan
    specific_humidity_2m_above_ground     (time, y, x) float64 3kB 0.009321 ....
    relative_humidity_2m_above_ground     (time, y, x) float64 3kB 66.0 ... nan
    u_component_of_wind_10m_above_ground  (time, y, x) float64 3kB 0.1141 ......
    v_component_of_wind_10m_above_ground  (time, y, x) float64 3kB 6.791 ... nan
    precipitable_water_entire_atmosphere  (time, y, x) float64 3kB 22.16 ... nan
Attributes:
    TIFFTAG_XRESOLUTION:     1
    TIFFTAG_YRESOLUTION:     1
    TIFFTAG_RESOLUTIONUNIT:  

...

Requesting data:   0%|          | 0/96 [02:20<?, ?it/s]
/Users/dhruvyadav/anaconda3/envs/NWS/lib/python3.10/site-packages/wxee/utils.py:149: UserWarning: Different non-null values were encountered for the same variable at the same time coordinate. The first value was taken.
  warnings.warn(


<xarray.Dataset> Size: 19kB
Dimensions:                               (time: 4, x: 10, y: 10)
Coordinates:
  * time                                  (time) datetime64[ns] 32B 2024-02-0...
  * x                                     (x) float64 80B -98.93 ... -96.91
  * y                                     (y) float64 80B 31.1 30.88 ... 29.08
    spatial_ref                           int64 8B 0
Data variables:
    temperature_2m_above_ground           (time, y, x) float64 3kB 13.41 ... nan
    specific_humidity_2m_above_ground     (time, y, x) float64 3kB 0.005923 ....
    relative_humidity_2m_above_ground     (time, y, x) float64 3kB 59.2 ... nan
    u_component_of_wind_10m_above_ground  (time, y, x) float64 3kB -0.0144 .....
    v_component_of_wind_10m_above_ground  (time, y, x) float64 3kB -3.743 ......
    precipitable_water_entire_atmosphere  (time, y, x) float64 3kB 12.78 ... nan
Attributes:
    TIFFTAG_XRESOLUTION:     1
    TIFFTAG_YRESOLUTION:     1
    TIFFTAG_RESOLUTIONUNIT:  

Requesting data:   0%|          | 0/96 [02:29<?, ?it/s]


<xarray.Dataset> Size: 19kB
Dimensions:                               (time: 4, x: 10, y: 10)
Coordinates:
  * time                                  (time) datetime64[ns] 32B 2024-02-0...
  * x                                     (x) float64 80B -98.93 ... -96.91
  * y                                     (y) float64 80B 31.1 30.88 ... 29.08
    spatial_ref                           int64 8B 0
Data variables:
    temperature_2m_above_ground           (time, y, x) float64 3kB 15.74 ... nan
    specific_humidity_2m_above_ground     (time, y, x) float64 3kB 0.004907 ....
    relative_humidity_2m_above_ground     (time, y, x) float64 3kB 42.0 ... nan
    u_component_of_wind_10m_above_ground  (time, y, x) float64 3kB -4.282 ......
    v_component_of_wind_10m_above_ground  (time, y, x) float64 3kB 1.867 ... nan
    precipitable_water_entire_atmosphere  (time, y, x) float64 3kB 9.559 ... nan
Attributes:
    TIFFTAG_XRESOLUTION:     1
    TIFFTAG_YRESOLUTION:     1
    TIFFTAG_RESOLUTIONUNIT:  

/Users/dhruvyadav/anaconda3/envs/NWS/lib/python3.10/site-packages/wxee/utils.py:149: UserWarning: Different non-null values were encountered for the same variable at the same time coordinate. The first value was taken.
  warnings.warn(
